# Tabulate rho(P,T) using Van der Waals equation of state
This notebook is based on code originally written by Claire Guimond

In [18]:
import numpy as np
import os
from numpy.polynomial import Polynomial
from thermotools.plot import *
from thermotools import moles
from thermotools import phase
from thermotools import get_inpdir, get_gendir, empty_dir
from matplotlib.ticker import MultipleLocator

In [19]:
elem_table = moles.read_elements()

datdir = os.path.join(get_gendir(), "vdw", "dat")
empty_dir(datdir)
pltdir = os.path.join(get_gendir(), "vdw", "plt")
empty_dir(pltdir)

Define constants

In [20]:
Rgas = 8.31446261815324

# Van Der Waals coefficients
#    From Wikipedia data page.
#    https://en.wikipedia.org/wiki/Van_der_Waals_constants_(data_page)
#    Units: bars, moles/litre
vdw = {
    "NH3"    : (4.225   ,  0.0371),
    "Ar"     : (1.355   ,  0.03201),
    "C4H10"  : (14.66   ,  0.1226),
    "CO2"    : (3.640   ,  0.04267),
    "CS2"    : (11.77   ,  0.07685),
    "CO"     : (1.505   ,  0.0398500),
    "CCl4"   : (19.7483 ,  0.1281),
    "Cl"     : (6.579   ,  0.05622),
    "C2N2"   : (7.769   ,  0.06901),
    "C4H10O" : (17.61   ,  0.1344),
    "CH3SCH3": (13.04   ,  0.09213),
    "C2H6"   : (5.562   ,  0.0638),
    "C2H5OH" : (12.18   ,  0.08407),
    "F"      : (1.171   ,  0.0290),
    "He"     : (0.0346  ,  0.0238),
    "N2H4"   : (8.46    ,  0.0462),
    "H2"     : (0.2476  ,  0.02661),
    "HBr"    : (4.510   ,  0.04431),
    "HCl"    : (3.716   ,  0.04081),
    "HCN"    : (11.29   ,  0.0881),
    "HF"     : (9.565   ,  0.0739),
    "HI"     : (6.309   ,  0.0530),
    "H2Se"   : (5.338   ,  0.04637),
    "H2S"    : (4.490   ,  0.04287),
    "C4H10"  : (13.32   ,  0.1164),
    "Kr"     : (2.349   ,  0.03978),
    "Kr"     : (8.200   ,  0.01696),
    "CH4"    : (2.253   ,  0.04278),
    "CH4O"   : (9.649   ,  0.06702),
    "Ne"     : (0.2135  ,  0.01709),
    "NO"     : (1.358   ,  0.02789),
    "N2"     : (1.370   ,  0.0387),
    "NO2"    : (5.354   ,  0.04424),
    "NF3"    : (3.58    ,  0.0545),
    "N2O"    : (3.832   ,  0.04415),
    "O2"     : (1.382   ,  0.03186),
    "O3"     : (3.570   ,  0.0487),
    "PH3"    : (4.692   ,  0.05156),
    "SiH4"   : (4.377   ,  0.05786),
    "SO2"    : (6.803   ,  0.05636),
    "SF6"    : (7.857   ,  0.0879),
    "CCl4"   : (20.01   ,  0.1281),
    "H2O"    : (5.536   ,  0.03049),
    "Xe"     : (4.250   ,  0.05105),
} 

# function to convert coefficients to SI
def coeff_SI(a, b):
    # Convert coefficients to SI
    a_SI = a / 1.e6 * 1.e5  # m^6.Pa.mol^-2
    b_SI = b / 1.e3  # m^3.mol^-1
    return a_SI, b_SI

# convert all of the coefficients to SI
for k in vdw.keys():
    a_SI, b_SI = coeff_SI(vdw[k][0], vdw[k][1])
    vdw[k] = (a_SI, b_SI)

Functions for evaluating VdW and Ideal equations of state

In [21]:
def rho_ideal(M, T, p):
    # SI units
    # ideal gas density in kg/m3
    return p * M / (Rgas * T)

def calc_critical_point(a, b):
    # SI units
    # https://www.thermopedia.com/content/1232/
    Tc = 8*a / (27*Rgas*b)
    pc = a / (27*b**2)
    return pc, Tc

def rho_VDW(a, b, M, T, p):
    """
    Evaluate density using Van der Waals equation of state.
        Input a and b in SI units, T in K, p in Pa, M in kg/mol.
        Returns density in kg/m3.
        https://scipython.com/book/chapter-6-numpy/problems/p64/the-van-der-waals-equation-of-state/
    """
    pc, Tc = calc_critical_point(a, b)

    def get_poly(a_SI, b_SI, T, p):
        return Polynomial([-a_SI * b_SI, a_SI, -(p * b_SI + Rgas * T), p])

    poly = get_poly(a, b, T, p)
    roots = poly.roots()
    roots.sort()
    n_real_roots = len(roots[roots.imag == 0])

    if n_real_roots == 1:
        Vgas = roots[roots.imag == 0][0].real  # in m3/mol
        return M  / Vgas
    elif n_real_roots == 3:
        # If below the critical point, we expect three roots, from which we want the smallest and largest:
        assert (T < Tc) or (p < pc)
        Vliq, Vgas = roots[0], roots[-1]  # in m3/mol
        return M / Vgas  # kg/m3
    else:
        raise NotImplementedError

Define a grid of P,T

In [22]:
p_eval = np.logspace(0,   10,   200)  # Pa
t_eval = np.arange(  105,  4000, 1.0)   # K
print("Num points = %g"%(len(p_eval)*len(t_eval),))

Num points = 779000


Evaluate density for every P,T point for each gas

In [23]:
# For each gas, create lookup table and plot
for gas in vdw.keys():
    print(gas)  

    a = vdw[gas][0]
    b = vdw[gas][1]
    M = moles.mmw_from_formula(gas, elem_table)

    # critical point 
    pc, Tc = calc_critical_point(a, b)
    print("    T_crit = %.3f K"%Tc)

    # create table 
    print("    generate data")
    rho_arr = [] # vdw gas density
    idl_arr = [] # ideal gas density
    tmp_arr = []
    prs_arr = []
    for p in p_eval:
        for t in t_eval:
            tmp_arr.append(t)
            prs_arr.append(p)
            rho_arr.append(rho_VDW(a, b, M, t, p))
            idl_arr.append(rho_ideal(M, t, p))

    # save to file 
    X = np.array([tmp_arr, np.log10(prs_arr), np.log10(rho_arr)]).T
    fpath = os.path.join(datdir, gas+".csv")
    np.savetxt(fpath, X, fmt="%.6e", header="T [K], log P [Pa], log rho [kg/m^3]", delimiter=',')    

    # plot 
    print("    plot")
    fig1, ax1 = plt.subplots(1, 1)
    fig2, ax2 = plt.subplots(1, 1)

    for ax in (ax1, ax2):
        ax.invert_yaxis()
        ax.set_yscale('log')
        ax.set_xlabel('T [K]')
        ax.set_ylabel('P [Pa]')
        ax.scatter(Tc, pc, marker='*', zorder=100, c='r')  # critical point

    shape = (len(p_eval), len(t_eval))
    P = np.reshape(prs_arr, shape )
    T = np.reshape(tmp_arr, shape )
    R = np.reshape(rho_arr, shape )
    I = np.reshape(idl_arr, shape )

    im1 = ax1.pcolormesh(T, P, R, cmap='viridis')
    fig1.colorbar(im1, label=r'$\rho$ [kg m-3]')
    fig1.savefig(os.path.join(pltdir, gas+"_rho.png"), bbox_inches='tight', dpi=140)

    ratio = R/I - 1
    vmax = np.amax(ratio)
    im2 = ax2.pcolormesh(T, P, ratio, cmap='RdBu', norm='symlog', vmin=-vmax, vmax=vmax)
    fig2.colorbar(im2, label=r'$(\rho_{\rm VDW} / \rho_{\rm ideal}) - 1$')
    fig2.savefig(os.path.join(pltdir, gas+"_ratio.png"), bbox_inches='tight', dpi=140)
    plt.close("all")

print("Done")

NH3
    T_crit = 405.831 K
    generate data
    plot
Ar
    T_crit = 150.850 K
    generate data
    plot
C4H10
    T_crit = 407.796 K
    generate data
    plot
CO2
    T_crit = 303.998 K
    generate data
    plot
CS2
    T_crit = 545.789 K
    generate data
    plot
CO
    T_crit = 134.586 K
    generate data
    plot
CCl4
    T_crit = 556.660 K
    generate data
    plot
Cl
    T_crit = 417.024 K
    generate data
    plot
C2N2
    T_crit = 401.185 K
    generate data
    plot
C4H10O
    T_crit = 466.930 K
    generate data
    plot
CH3SCH3
    T_crit = 504.392 K
    generate data
    plot
C2H6
    T_crit = 310.672 K
    generate data
    plot
C2H5OH
    T_crit = 516.295 K
    generate data
    plot
F
    T_crit = 143.897 K
    generate data
    plot
He
    T_crit = 5.181 K
    generate data
    plot
N2H4
    T_crit = 652.560 K
    generate data
    plot
H2
    T_crit = 33.159 K
    generate data
    plot
HBr
    T_crit = 362.716 K
    generate data
    plot
HCl
    T_crit = 324.4